In [1]:
# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
from pandas import DataFrame
import numpy as np
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# establece el estilo del fondo de las gráficas como "whitegrid". Las gráficas tendrán un fondo blanco con líneas de cuadrícula horizontales y verticales.
sns.set_style("whitegrid")

# cambiará el esquema de colores predeterminado de Seaborn a uno más oscuro, adecuado para gráficas con fondos claros. 
sns.set_theme(style="dark")

# establece el estilo en Solarize_Lihgt2. 
plt.style.use('Solarize_Light2')

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

import scipy.stats as stats

# -----------------------------------------------------------------------
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import levene

import mysql.connector
from mysql.connector import errorcode

In [2]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [16]:
df = pd.read_csv('per-capita-mismanaged-plastic-waste-vs-gdp-per-capita.csv')

In [17]:
df.shape

(48168, 7)

In [18]:
df.columns

Index(['Entity', 'Code', 'Year', 'Per capita mismanaged plastic waste',
       'GDP per capita, PPP (constant 2011 international $)',
       'Total population (Gapminder, HYDE & UN)', 'Continent'],
      dtype='object')

In [21]:
df_mismanage = df[df['Year']==2010]
df_mismanage=df_mismanage[df_mismanage['GDP per capita, PPP (constant 2011 international $)'].notna()&df_mismanage['Per capita mismanaged plastic waste'].notna() ]
df_mismanage_ok = df_mismanage[['Entity','Year','Per capita mismanaged plastic waste','GDP per capita, PPP (constant 2011 international $)','Total population (Gapminder, HYDE & UN)']]

In [22]:
df_mismanage_ok

,Entity,Year,Per capita mismanaged plastic waste,"GDP per capita, PPP (constant 2011 international $)","Total population (Gapminder, HYDE & UN)"
344,Albania,2010,0.032,9927.181841,2.948000e+06
564,Algeria,2010,0.086,12870.602699,3.597700e+07
1074,Angola,2010,0.045,5897.682841,2.335600e+07
1365,Antigua and Barbuda,2010,0.051,19212.720131,8.800000e+04
1613,Argentina,2010,0.026,18712.063077,4.089600e+07
2245,Australia,2010,0.002,41464.040070,2.215500e+07
2907,Bahamas,2010,0.011,29221.991182,3.550000e+05
3127,Bahrain,2010,0.016,40570.535693,1.241000e+06
3347,Bangladesh,2010,0.030,2442.728888,1.475750e+08
3567,Barbados,2010,0.034,16417.962452,2.820000e+05


In [16]:
df_verde.sample()

Entity
Marshall Islands    False
Name: Per capita mismanaged plastic waste, dtype: bool

In [6]:
df.sample(5).T

Entity,Mali,Greece,Montserrat,Armenia,Jamaica
Code,MLI,GRC,MSR,ARM,JAM
Year,1846,1834,1975,1953,1986
Per capita mismanaged plastic waste,NaN,NaN,NaN,NaN,NaN
"GDP per capita, PPP (constant 2011 international $)",NaN,NaN,NaN,NaN,NaN
"Total population (Gapminder, HYDE & UN)",1785159.0,2632714.0,12000.0,1461000.0,2358000.0
Continent,NaN,NaN,NaN,NaN,NaN


In [65]:
def exploracion_inicial(df):
    """Esta funcion hace una exploracion inicial de las columnas de un dataframe
        Parametros:
        df ... dataframe 
        return:
        nada, solamente lo analizamos y lo printeamos por pantalla para ir tomando decisiones"""
    for columna in df.columns:
        print('*********************************')
        print(f'Analizando la columna {columna}:')
        print(f'La columna es de tipo {df[columna].dtypes}')
        
        print(f'El porcentaje de nulos de esta columna es de  {round(df[columna].isnull().sum() / df.shape[0] * 100,2)} %')
        print(f'Los valores unicos son: {df[columna].value_counts()}')
        print(f'Los composicion de la columna por valores es : { df[columna].value_counts()}')


In [66]:


def dataframe_nulos(df):
    """Esta funcion coge un dataframe y una lista de columnas y crea otro dataframe con informacion resultante"""
    
    # Creamos un diccionario vacío para almacenar los resultados
    resultados = {}
    
    for columna in df.columns:
        # Calcular el porcentaje de valores nulos para la columna actual
        porcentaje_nulos = round((df[columna].isnull().sum() / df.shape[0]) * 100,2)
        
        # Obtener el tipo de datos de la columna actual
    
        tipo = df[columna].dtypes
        
        # Agregar la información al diccionario de resultados
        resultados[columna] = {"Tipo": tipo, "%nulos": porcentaje_nulos}

    # Creamo un dataFrame a partir del diccionario
    df_resultado = pd.DataFrame(resultados).T
    print(df_resultado)







In [67]:
dataframe_nulos(df)

                                             Tipo %nulos
Code                                       object    0.0
Year                                        int64    0.0
Per capita plastic waste (kg/person/day)  float64    0.0


In [49]:
#Columnas numéricas con nulos 
nulos_num = df[df.columns[df.isnull().any()]].select_dtypes(include = np.number).columns
print(nulos_num)


Index(['Per capita plastic waste (kg/person/day)',
       'GDP per capita, PPP (constant 2011 international $)',
       'Total population (Gapminder, HYDE & UN)'],
      dtype='object')


In [50]:
df_year = df.copy()
df_year = df[df['Year']==2010]
#df_year.drop(['Per capita plastic waste (kg/person/day)','GDP per capita, PPP (constant 2011 international $)','Continent'],axis=1,inplace=True)


In [53]:
df_year.columns

Index(['Code', 'Year', 'Per capita plastic waste (kg/person/day)',
       'GDP per capita, PPP (constant 2011 international $)',
       'Total population (Gapminder, HYDE & UN)', 'Continent'],
      dtype='object')

In [54]:
df_year.shape

(292, 6)

In [ ]:
df_year

In [59]:
df_year['Per capita plastic waste (kg/person/day)'].isnull().sum()

106